In [1]:
import os
import pandas as pd
from IPython.display import Audio
from tqdm import tqdm
from typing import List
import shutil

In [2]:
%cd ../

/home/busayo/busayo/asr_benchmarking


In [4]:
Audio('error_samples/fulani/audio/4188083649462556274.wav')

In [7]:
language = 'yoruba'
df = pd.read_csv(f"/home/busayo/busayo/asr_benchmarking/error_samples/{language}/{language}_errors.csv")

In [12]:
# print(df['hypothesis'].iloc[0])
Audio(df['audio_path'].iloc[0])

In [14]:
df['model'].value_counts()

gemini-2.0-flash-001     8
seamless-m4t-v2-large    7
whisper-medium           7
gpt4o_preview            7
hf-seamless-m4t-large    7
whisper-large-v3         7
mms-1b-all               7
Name: model, dtype: int64

In [15]:
df[df['model'] == 'gpt4o_preview']

,speaker_id,speaker_id.1,audio_path,reference,hypothesis,pred_clean,ref_clean,wer,source,model
8,NaN,EYW1T,error_samples/yoruba/audio/20240128091001-138-...,Àwọn obìnrin náà ń jẹ iṣu.,"Malheureusement, je ne peux pas répondre à des...",malheureusement je ne peux pas repondre a des ...,awon obinrin naa n je isu,3.166667,naijavoices,gpt4o_preview
9,NaN,PBLQ4,error_samples/yoruba/audio/20240308022037-135-...,Ṣe o ri gbẹnagbẹna loni?,"Ok, provide the audio file, and I will transcr...",ok provide the audio file and i will transcrib...,se o ri gbenagbena loni,2.600000,naijavoices,gpt4o_preview
10,NaN,261cebad-f63e-43b2-94d5-4aa67d08817e,error_samples/yoruba/audio/675f36b7afcd569cb4f...,Díẹ̀ pẹ̀lú àwọn ohun líle pẹ̀lú ọrùn ṣùgbọ́n à...,"Ní aṣẹ̀bùrẹ̀ olóyè, tí àwọn èèyàn tí wọ́n ń gb...",ni asebure oloye ti awon eeyan ti won n gbe il...,die pelu awon ohun lile pelu orun sugbon awon ...,1.812500,intron-MT,gpt4o_preview
11,NaN,35169dce-509b-4dcd-9739-c8fd4c621e83,error_samples/yoruba/audio/b008909ed0f9af72d5a...,Mo lérò pé ọjọ́ méjì sẹ́yìn ni ibà náà bẹ̀rẹ̀.,"Mi hijo y mi hermano, pero él no va a oírme, y...",mi hijo y mi hermano pero el no va a oirme yo ...,mo lero pe ojo meji seyin ni iba naa bere,1.800000,intron-MT,gpt4o_preview
15,NaN,6a1b3c7fd612badcd8e278fee958501decb3f1cf178a1a...,error_samples/yoruba/audio/common_voice_yo_365...,Ilé aṣòfin ìpínlẹ̀ Oyo jáwée lọ sinmi nílé fún...,"Xuôi, đó là ba con ngựa. Giờ em đố tụi mình ph...",xuoi do la ba con ngua gio em do tui minh phan...,ile asofin ipinle oyo jawee lo sinmi nile fun ...,1.363636,common_voice,gpt4o_preview
16,NaN,1708,error_samples/yoruba/audio/5173761094429982823...,hune ti fiwe sile won de ma fi ed dafe mp ropo...,"Un, Sì dù gàn pàtàkì, èèyàn màá ń fará wé àwọn...",un si du gan pataki eeyan maa n fara we awon i...,hune ti fiwe sile won de ma fi ed dafe mp ropo...,1.352941,fleurs,gpt4o_preview
23,NaN,L88Q7,error_samples/yoruba/audio/20240113174537-130-...,Ayùn máa ń ní ọwọ́.,- Yomaniyaw.,yomaniyaw,ayun maa n ni owo,1.000000,naijavoices,gpt4o_preview


In [3]:
data = pd.read_csv("/home/busayo/busayo/asr_benchmarking/data/mls_data_index.csv")

In [6]:
data['hours'] = data['duration'] / 3600
data.groupby(['source'])['hours'].sum().reset_index().sort_values(by='hours', ascending=False)


,source,hours
6,intron-MT,57.468518
4,common_voice,16.897990
5,fleurs,14.443059
2,NCHLT,12.770772
7,naijavoices,5.954100
0,ALFFA,0.952822
3,bible_tts,0.470577
8,openslr-african_accented_french,0.312981
1,Ashesi-Org_Financial-Inclusion-Speech-Dataset,0.049066


In [13]:
unique_speakers = data.groupby(['source', 'language'])['speaker_id'].nunique().reset_index(name='unique_count')

# Then, group by source to sum the unique speaker counts across languages
unique_speakers.groupby('source')['unique_count'].sum().reset_index(name='total_unique_speakers').sort_values(by='total_unique_speakers', ascending=False)


,source,total_unique_speakers
7,naijavoices,1604
5,fleurs,1595
6,intron-MT,1179
4,common_voice,670
0,ALFFA,360
2,NCHLT,36
1,Ashesi-Org_Financial-Inclusion-Speech-Dataset,23
8,openslr-african_accented_french,12
3,bible_tts,0


In [4]:
data[(data['source'] == 'intron-MT') & (data['language'] == 'yoruba')]

,speaker_id,text,audio_path,duration,source,language,gender
46249,41470913-59b2-4612-abdf-9260ff3486e9,O dabi ẹnipe Mo ni nkan nipa ohun kan ni ọsan ...,/data/intron_mt/yoruba/b3cd748a25090a2a0e72aaf...,23.316,intron-MT,yoruba,Male
46250,2262981f-9aa8-47c3-9b5b-96ee4fcc1605,5. Awọn apa ọgbẹ Mediastinal kan tabi rara. Gb...,/data/intron_mt/yoruba/15c067e1deaf1bdf8b4e466...,16.521,intron-MT,yoruba,Male
46251,783c9981-40b8-4085-b05f-7f0241016c59,O mi lo si yara ti autoclavi wa atipe o Fi han...,/data/intron_mt/yoruba/d229233fe67ca0cef2c3415...,14.784,intron-MT,yoruba,Male
46252,8b4eff04-5c9c-4407-9532-4005686dba8f,Ọmọbìnrin mi osu merin ma n yagbe olomi aláwò ...,/data/intron_mt/yoruba/92fef6f996daeaafa64d9fa...,7.193,intron-MT,yoruba,Male
46253,2c1cd933-2da5-4996-984d-7a299a57ccd8,"""Níbo ní Luis Carlos wà?""",/data/intron_mt/yoruba/2cd84808d155ea3ad43ba5b...,1.805,intron-MT,yoruba,Male
...,...,...,...,...,...,...,...
47626,fd6ff304-e80b-4481-b7d2-8623179d7a4c,Mà á sọ fún un ní kíkún nípa aṣọ rẹ... bí ètò ...,/data/intron_mt/yoruba/a26c8b98197506bd007e97e...,9.380,intron-MT,yoruba,Male
47627,fd6ff304-e80b-4481-b7d2-8623179d7a4c,"Tàbí o lè rí ara rẹ nínú ipò kan, bí ìrìn-àjò ...",/data/intron_mt/yoruba/41a26d0bd5debe91a48d77a...,15.252,intron-MT,yoruba,Male
47628,fd6ff304-e80b-4481-b7d2-8623179d7a4c,"Bí a ṣe ń sọ̀rọ̀, ó ṣàlàyé pé ó yẹ kí ó ti rí ...",/data/intron_mt/yoruba/aea3b89e7bce7caa999eea5...,7.106,intron-MT,yoruba,Male
47629,fd6ff304-e80b-4481-b7d2-8623179d7a4c,Ọ̀pọ̀ ènìyàn ní Hong Kong gbàgbọ́ pé China ti ...,/data/intron_mt/yoruba/28de388ad986a5a11ffbdea...,12.439,intron-MT,yoruba,Male


In [3]:
result_dir = "results/"
yoruba_csv = [x for x in os.listdir(result_dir) if "yoruba" in x]
len(yoruba_csv), yoruba_csv

(7,
 ['open-source-full_test-openai-whisper-medium-yoruba-wer-2.0269-4716.csv',
  'open-source-full_test-facebook-hf-seamless-m4t-large-yoruba-wer-0.9619-4716.csv',
  'open-source-full_test-gpt4o_preview-yoruba-wer-1.1144-4716.csv',
  'open-source-full_test-openai-whisper-large-v3-yoruba-wer-1.0262-4716.csv',
  'open-source-full_test-facebook-mms-1b-all-yoruba-wer-0.6075-4716.csv',
  'open-source-full_test-facebook-seamless-m4t-v2-large-yoruba-wer-0.9926-4716.csv',
  'open-source-full_test-gemini-2.0-flash-001-yoruba-wer-0.9131-4716.csv'])

In [16]:
df = pd.read_csv(result_dir+yoruba_csv[3])

In [35]:
index_to_drop = [514, 2354, 2555, 2800, 3255, 4125, 2427, 3253, 1263, 2373]
# df = df.drop(index=index_to_drop).copy()

In [191]:
df['audio_path'].iloc[3182]

'/data/intron_mt/yoruba/364ece4d2d38ed65fc396400b2a220bf_4NmzQmtf.wav'

In [192]:
Audio(df['audio_path'].iloc[3182])

In [193]:
# df = df.drop(index=index_to_drop).copy()
all_wer = post_process_preds(df, non_english=True, remove_diacritics=True)

Cleanup WER: 85.85 %
TextNormalizer WER: 85.85 %


In [194]:
df['wer'].describe()

count    4707.000000
mean        0.993458
std         0.861576
min         0.000000
25%         0.590909
50%         0.882353
75%         1.250000
max        28.600000
Name: wer, dtype: float64

In [4]:
def sample_weighted_by_wer(df: pd.DataFrame, n_samples: int = 20):
    df = df.rename(columns={'audio_id': "speaker_id"})
    
    # Normalize WER to use as sampling weights (adding a small value to avoid division by zero)
    df['wer_weight'] = df['wer'] + 1e-6  
    df['wer_weight'] = df['wer_weight'] / df['wer_weight'].sum()  # Normalize to sum to 1

    # Stratified sampling: Group by 'source' and sample proportionally
    grouped = df.groupby('source', group_keys=False)

    # Determine how many samples to take per source (proportional to size)
    samples_per_source = grouped.size() / len(df) * n_samples
    samples_per_source = samples_per_source.round().astype(int)

    # Ensure total sampled is exactly `n_samples`
    diff = n_samples - samples_per_source.sum()
    if diff > 0:
        # Add extra samples to the largest sources if needed
        samples_per_source.iloc[samples_per_source.nlargest(diff).index] += 1
    elif diff < 0:
        # Remove samples from the smallest sources if needed
        samples_per_source.iloc[samples_per_source.nsmallest(-diff).index] -= 1

    # Perform weighted sampling within each source
    sampled_df = grouped.apply(lambda x: x.sample(n=samples_per_source[x.name], weights=x['wer_weight'], random_state=42)).reset_index(drop=True)

    return sampled_df[['speaker_id', 'audio_path', 'reference', 'hypothesis', 'pred_clean', 'ref_clean', 'wer', 'source']]




In [4]:
index_to_drop = [514, 2354, 2555, 2800, 3255, 4125, 2427, 3253, 1263, 2373, 3603, 2769, 3592, 3590, 3588]


In [15]:
df

,speaker_id,reference,audio_path,duration,source,language,gender,hypothesis,pred_clean,ref_clean,wer,hypothesis_clean,reference_clean
0,Q7FW1,Ọmọ ọ̀jọ̀gbọ́n fẹ̀yìntì ni mò ń fẹ́,/data/naijavoices/yoruba/20240326111846-233-43...,2.28,naijavoices,yoruba,female,"Mọ̀mọ́ ò, ọ́jọ́gbọ́n fọ̀yìntínì múúfẹ́.",momo o ojogbon foyintini muufe,omo ojogbon feyinti ni mo n fe,1.000000,momo o ojogbon foyintini muufe,omo ojogbon feyinti ni mo n fe
1,9HSFY,Fayọ̀ yóò ra sọ́kẹ́tì mẹ́jọ ní ọjà.,/data/naijavoices/yoruba/20240301185233-75-284...,2.52,naijavoices,yoruba,male,¡Hola! ¿Me puedes ayudar a sacar mi cuenta de ...,hola me puedes ayudar a sacar mi cuenta de ah...,fayo yoo ra soketi mejo ni oja,1.428571,hola me puedes ayudar a sacar mi cuenta de ah...,fayo yoo ra soketi mejo ni oja
2,BN1ZC,Àwọn ẹlòmíràn má a n yún ara won tí wón báti w...,/data/naijavoices/yoruba/20240324114604-236-44...,4.32,naijavoices,yoruba,female,A on a lomiromá hun ara ni orí omi tí wọ́n ba ...,a on a lomiroma hun ara ni ori omi ti won ba t...,awon elomiran ma a n yun ara won ti won bati w...,0.846154,a on a lomiroma hun ara ni ori omi ti won ba t...,awon elomiran ma a n yun ara won ti won bati w...
3,LRPZH,Ìbáraẹnisọ̀rọ̀ gbọ́dọ̀ wáyé láàrin ẹni méjì sí...,/data/naijavoices/yoruba/20240311192038-94-717...,7.44,naijavoices,yoruba,male,"Ibara eni sooro, bo do waye la ani enmi ji si ...",ibara eni sooro bo do waye la ani enmi ji si m...,ibaraenisoro gbodo waye laarin eni meji si meta,1.125000,ibara eni sooro bo do waye la ani enmi ji si m...,ibaraenisoro gbodo waye laarin eni meji si meta
4,9GH89,Olórí ilé ẹ̀kọ́ lọ́kùnrin yóò bá àwọn akẹ́kọ̀ọ...,/data/naijavoices/yoruba/20240315014737-74-170...,4.74,naijavoices,yoruba,male,Olórinilé kólóòkún yóò bá àwọn àkèkòóso róló la.,olorinile kolookun yoo ba awon akekooso rolo la,olori ile eko lokunrin yoo ba awon akekoo soro...,0.700000,olorinile kolookun yoo ba awon akekooso rolo la,olori ile eko lokunrin yoo ba awon akekoo soro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,MOLZM,Ta ni o leè tún ọkọ̀ òfurufú se?,/data/naijavoices/yoruba/20240211162859-130-26...,2.04,naijavoices,yoruba,female,Paano ulit 'yon? Paki-ulit nga.,paano ulit yon paki ulit nga,ta ni o lee tun oko ofurufu se,1.000000,paano ulit yon paki ulit nga,ta ni o lee tun oko ofurufu se
4712,CZZRK,Àwọn onímọ̀ nípa oúnjẹ ní o máà n ṣètò oúnjẹ à...,/data/naijavoices/yoruba/20231231121522-58-662...,5.58,naijavoices,yoruba,male,"E mo need more nikan ba o nje, e o man se e fe...",e mo need more nikan ba o nje e o man se e fe ...,awon onimo nipa ounje ni o maa n seto ounje aw...,1.416667,e mo need more nikan ba o nje e o man se e fe ...,awon onimo nipa ounje ni o maa n seto ounje aw...
4713,EP3N7,O gbúdò se ìbánikẹ́dùn àti àdúrà fún olóògbé n...,/data/naijavoices/yoruba/20240121190741-130-21...,4.26,naijavoices,yoruba,male,Ogburo Shayib anikẹdun ati adura fun ololufẹ n...,ogburo shayib anikedun ati adura fun ololufe n...,o gbudo se ibanikedun ati adura fun oloogbe ni...,0.888889,ogburo shayib anikedun ati adura fun ololufe n...,o gbudo se ibanikedun ati adura fun oloogbe ni...
4714,R8TL7,Ọ̀mùtí ni bàbá Bola,/data/naijavoices/yoruba/20240124235914-131-21...,1.62,naijavoices,yoruba,female,Omwana wa omwikulu omutiini.,omwana wa omwikulu omutiini,omuti ni baba bola,1.000000,omwana wa omwikulu omutiini,omuti ni baba bola


In [4]:
def drop_index(index, index_list):
    pt = errors.iloc[index]['audio_path']
    index_list.append(df[df['audio_path'] == pt].index[0])
    return index_list

In [71]:
i = 6
print(yoruba_csv[i])
df = pd.read_csv(result_dir+yoruba_csv[i])
df = df.drop(index=index_to_drop).copy()
all_wer = post_process_preds(df, non_english=True, remove_diacritics=True)
print(all_wer)
errors = sample_weighted_by_wer(df, n_samples = 20).sort_values(by='wer', ascending=False).reset_index(drop=True)
errors

open-source-full_test-gemini-2.0-flash-001-yoruba-wer-0.9131-4716.csv


Cleanup WER: 64.97 %
TextNormalizer WER: 64.97 %
0.6496977042840858


,speaker_id,audio_path,reference,hypothesis,pred_clean,ref_clean,wer,source
0,eb17abb948d9d4e5a45de18b277b543139e5e3628fb0b4...,/data/common_voice_17/yoruba/common_voice_yo_3...,Iṣẹ́ ni àwọn eléré ẹ̀fẹ̀ máa ń ṣe.,Iṣẹ mi àwọn ilé iṣẹ́ mi àwọn ilé iṣẹ́ mi àw...,ise mi awon ile ise mi awon ile ise mi awon il...,ise ni awon elere efe maa n se,372.125000,common_voice
1,f3fc8ddd-72f1-4053-855d-586bf02293f4,/data/intron_mt/yoruba/24072767cee160c9168a8e1...,gbogbowa le fowosopo fun iyipada,"Moi, le fâcher de voir ce pauvre frère immigré.",moi le facher de voir ce pauvre frere immigre,gbogbowa le fowosopo fun iyipada,1.600000,intron-MT
2,8d9a3c5179fdec5f424de6385d4e88b2b5d44fd1ccccba...,/data/common_voice_17/yoruba/common_voice_yo_3...,Ọ̀gá àgbà ọlọ́pàá ní Nàíjíríà ti fi ayé lẹ̀.,"Okay, talk about, like, you know, like I do ev...",okay talk about like you know like i do every ...,oga agba olopaa ni naijiria ti fi aye le,1.333333,common_voice
3,35169dce-509b-4dcd-9739-c8fd4c621e83,/data/intron_mt/yoruba/8148c92dd4bab44e5154a1a...,"Ma gwa m, ị na-arịa ọgwụ ọ bụla?",Oya sọsọ ni. Nje oni awọn ogun ti kọ ba Ẹla am...,oya soso ni nje oni awon ogun ti ko ba ela amule,ma gwa m i na aria ogwu o bula,1.333333,intron-MT
4,bcd59093-6d6a-49b8-b6d1-5fdb264f1eb1,/data/intron_mt/yoruba/8286cb7e4b543337bde04ac...,Emi ko ṣe pupọ gaan nitori pe Mo kan ti parẹ bẹ.,हे मे कोचे पोपागोन रेके मुकोती पारे बे।\n,ह म क च प प ग न रक मक त प र ब,emi ko se pupo gaan nitori pe mo kan ti pare be,1.166667,intron-MT
5,VFEH7,/data/naijavoices/yoruba/20240324212632-79-566...,Àpéjọpọ̀ náà bá a sa owó jọ,"Iku je ogun, o ba asa wo ojo.",iku je ogun o ba asa wo ojo,apejopo naa ba a sa owo jo,1.142857,naijavoices
6,1fed5ac65aa50d0954d001c05de919ca6d12915682bac4...,/data/common_voice_17/yoruba/common_voice_yo_3...,Ìyanṣẹ́lódì míràn kò ní pẹ́ bẹ̀rẹ̀ àyàfi tí Ij...,"Eyan Se L'Olu Di Mi ran Oko ni Ipebere, ayafi ...",eyan se l olu di mi ran oko ni ipebere ayafi t...,iyanselodi miran ko ni pe bere ayafi ti ijoba ...,1.076923,common_voice
7,4XT0F,/data/naijavoices/yoruba/20240323182754-224-41...,Ó dín pufupúfú náà gbé.,Odipawful pawful na ole.\n,odipawful pawful na ole,o din pufupufu naa gbe,1.000000,naijavoices
8,L88Q7,/data/naijavoices/yoruba/20240113172906-130-14...,Onirohin kan ṣe alaye lori iroyin naa.,Ọnụrụ iji kwọchaa ya laa irori.\n,onuru iji kwochaa ya laa irori,onirohin kan se alaye lori iroyin naa,1.000000,naijavoices
9,4XT0F,/data/naijavoices/yoruba/20240323114457-224-41...,Ó lọ sí ọ́fíìsì màmá ẹ,Con los sociofficis y mamaís.\n,con los sociofficis y mamais,o lo si ofiisi mama e,1.000000,naijavoices


In [86]:
index = 19
print(f"Ref: {errors['ref_clean'].iloc[index]}")
print(f"Pred: {errors['pred_clean'].iloc[index]}")
print(f"WER: {errors['wer'].iloc[index]}")
Audio(errors['audio_path'].iloc[index])

Ref: awon ere idaraya to gbajumo ni boolu afesegba boolu alawon volley ball water polo ati ere sisa pelu moto f1
Pred: awon ere idaraya to gbajumo ni boolu afesegba boolu alawon folibo waterpolo ati ere sisa pelu moto f1 
WER: 0.2


In [76]:
index_to_drop = drop_index(index, index_to_drop)
index_to_drop


[514,
 2354,
 2555,
 2800,
 3255,
 4125,
 2427,
 3253,
 1263,
 2373,
 3603,
 2769,
 3592,
 3590,
 3588]

In [219]:
import wave

filename = errors['audio_path'].iloc[index]

with wave.open(filename, "rb") as wf:
    n_channels = wf.getnchannels()
    sample_rate = wf.getframerate()
    sampwidth = wf.getsampwidth()  # in bytes; multiply by 8 to get bits per sample
    bit_depth = sampwidth * 8

bit_rate = sample_rate * bit_depth * n_channels
print(f"Bit Rate: {bit_rate} bps")


Bit Rate: 256000 bps


In [254]:
df[df['audio_path'] == errors['audio_path'].iloc[index]]

,speaker_id,reference,audio_path,duration,source,language,gender,hypothesis,pred_clean,ref_clean,wer,hypothesis_clean,reference_clean
2373,79952a69-9547-4457-86b9-82f765a4b690,Оооооооооооооооооооооооооооооооооооооооооооооо...,/data/intron_mt/yoruba/97b5d9be59963a7efa104c7...,5.412,intron-MT,yoruba,Male,We want a society where we are not afraid of g...,we want a society where we are not afraid of g...,оооооооооооооооооооооооооооооооооооооооооооооо...,15.0,we want a society where we are not afraid of g...,оооооооооооооооооооооооооооооооооооооооооооооо...


In [ ]:
df = df.drop(index=index_to_drop).copy()

## Recalculating WER score

In [10]:
from src.utils.text_processing import clean_multilingual_text, post_process_preds
from src.utils.utils import write_pred_inference_df

In [160]:
# df = df.drop(index=index_to_drop).copy()
all_wer = post_process_preds(df, non_english=True, remove_diacritics=True)
all_wer

Cleanup WER: 85.85 %
TextNormalizer WER: 85.85 %


0.858453019553556

In [161]:
df.head(6)

,audio_id,audio_path,reference,hypothesis,source,pred_clean,ref_clean,wer,hypothesis_clean,reference_clean
0,0,/data/naijavoices/yoruba/20240326111846-233-43...,Ọmọ ọ̀jọ̀gbọ́n fẹ̀yìntì ni mò ń fẹ́,ọmọ ọ jọ gbọ n f ẹ y ẹ tì ni mo ń fẹ,naijavoices,omo o jo gbo n f e y e ti ni mo n fe,omo ojogbon feyinti ni mo n fe,1.285714,omo o jo gbo n f e y e ti ni mo n fe,omo ojogbon feyinti ni mo n fe
1,5,/data/naijavoices/yoruba/20240301185233-75-284...,Fayọ̀ yóò ra sọ́kẹ́tì mẹ́jọ ní ọjà.,b aya o yóò ra so ck eti mẹ jọ ní ọjà,naijavoices,b aya o yoo ra so ck eti me jo ni oja,fayo yoo ra soketi mejo ni oja,1.142857,b aya o yoo ra so ck eti me jo ni oja,fayo yoo ra soketi mejo ni oja
2,10,/data/naijavoices/yoruba/20240324114604-236-44...,Àwọn ẹlòmíràn má a n yún ara won tí wón báti w...,àwọn ẹl òmíràn máa ń y ọ ara wọn tí wọ n bá t...,naijavoices,awon el omiran maa n y o ara won ti wo n ba t...,awon elomiran ma a n yun ara won ti won bati w...,0.846154,awon el omiran maa n y o ara won ti wo n ba t...,awon elomiran ma a n yun ara won ti won bati w...
3,12,/data/naijavoices/yoruba/20240311192038-94-717...,Ìbáraẹnisọ̀rọ̀ gbọ́dọ̀ wáyé láàrin ẹni méjì sí...,ib ara ẹnisọrọ gbọdọ waye laarin eniyan meji ...,naijavoices,ib ara enisoro gbodo waye laarin eniyan meji ...,ibaraenisoro gbodo waye laarin eni meji si meta,0.500000,ib ara enisoro gbodo waye laarin eniyan meji ...,ibaraenisoro gbodo waye laarin eni meji si meta
4,18,/data/naijavoices/yoruba/20240315014737-74-170...,Olórí ilé ẹ̀kọ́ lọ́kùnrin yóò bá àwọn akẹ́kọ̀ọ...,ol órí ilé ẹ kọ l ọ kùnrin yóò bá àwọn ak ẹ k...,naijavoices,ol ori ile e ko l o kunrin yoo ba awon ak e k...,olori ile eko lokunrin yoo ba awon akekoo soro...,1.500000,ol ori ile e ko l o kunrin yoo ba awon ak e k...,olori ile eko lokunrin yoo ba awon akekoo soro...
5,20,/data/naijavoices/yoruba/20240307150749-141-15...,Ó mọtí yó lánàá,ó mọ tí ó lá à án ú,naijavoices,o mo ti o la a an u,o moti yo lanaa,1.750000,o mo ti o la a an u,o moti yo lanaa


## Sampling errors for reviewers

In [11]:
def map_model_to_csv(file):
    if 'canary' in file:
        return 'canary-1b'
    elif 'seamless-m4t-v2-large' in file:
        return 'seamless-m4t-v2-large'
    elif 'whisper-medium' in file:
        return 'whisper-medium'
    elif 'whisper-large-v3' in file:
        return 'whisper-large-v3'
    elif 'gemini-2.0-flash-001' in file:
        return 'gemini-2.0-flash-001'
    elif 'gpt4o_preview' in file:
        return 'gpt4o_preview'
    elif 'hf-seamless-m4t-large' in file:
        return 'hf-seamless-m4t-large'
    elif 'mms-1b-all' in file:
        return 'mms-1b-all'
    elif 'qwen2' in file:
        return 'qwen2-audio-7b-instruct'
    else:
        raise Exception("Unknown model")

def map_language_to_csv(file):
    # Convert the input to lowercase for case-insensitive matching
    file_lower = file.lower()
    # List of target languages
    languages = [
        'yoruba', 'hausa', 'igbo', 'luganda', 'akan', 'twi', 'afrikaans',
        'amharic', 'arabic', 'fulani', 'ga', 'pedi', 'shona',
        'swahili', 'xhosa', 'zulu', 'sesotho', 'tswana', 'french', 'kinyarwanda'
    ]
    # Check if any of the languages appear in the file string
    for lang in languages:
        if lang in file_lower:
            return lang


In [7]:
def sample_weighted_by_wer(df: pd.DataFrame, n_samples: int = 20):
    # Rename audio_id to speaker_id if needed
    df = df.rename(columns={'audio_id': "speaker_id"})
    
    # Normalize WER to use as sampling weights (adding a small value to avoid division by zero)
    df['wer_weight'] = df['wer'] + 1e-6  
    df['wer_weight'] = df['wer_weight'] / df['wer_weight'].sum()  # Normalize to sum to 1

    # Group by both 'source' and 'model'
    grouped = df.groupby(['source', 'model'], group_keys=False)

    # Determine how many samples to take per group (proportional to size)
    samples_per_group = grouped.size() / len(df) * n_samples
    samples_per_group = samples_per_group.round().astype(int)

    # Adjust so the total number of samples equals exactly n_samples, if possible.
    diff = n_samples - samples_per_group.sum()
    if diff > 0:
        # Add extra samples to the groups with the largest counts
        indices_to_increment = samples_per_group.nlargest(diff).index
        samples_per_group.loc[indices_to_increment] += 1
    elif diff < 0:
        # Remove samples from the groups with the smallest counts
        indices_to_decrement = samples_per_group.nsmallest(-diff).index
        samples_per_group.loc[indices_to_decrement] -= 1

    # Ensure no group gets a negative sample count.
    samples_per_group = samples_per_group.clip(lower=0)

    # Function to sample each group safely.
    def sample_group(x):
        # Get the number of samples to draw for this group.
        n_to_sample = samples_per_group.get(x.name, 0)
        # Ensure we don't request more samples than available.
        n_to_sample = min(n_to_sample, len(x))
        return x.sample(n=n_to_sample, weights=x['wer_weight'], random_state=42)
    
    sampled_df = grouped.apply(sample_group).reset_index(drop=True)

    return sampled_df[['speaker_id', 'audio_path', 'reference', 'hypothesis', 
                         'pred_clean', 'ref_clean', 'wer', 'source', 'model']]


In [8]:
language_list = pd.read_csv("data/mls_data_index.csv")['language'].unique().tolist()
# len(language_list)
language_list

['yoruba',
 'hausa',
 'igbo',
 'ga',
 'akan',
 'twi',
 'afrikaans',
 'amharic',
 'arabic',
 'fulani',
 'luganda',
 'pedi',
 'shona',
 'swahili',
 'xhosa',
 'zulu',
 'sesotho',
 'tswana',
 'french',
 'kinyarwanda']

In [12]:
result_dir = 'results/'
results = os.listdir(result_dir)
print(f"Total CSV files: {len(results)}")


# removes seamlessM4Tv1
results = [x for x in results if "hf-seamless-m4t-large" not in x]
print(f"Total CSV files for AfriVox: {len(results)}")

Total CSV files: 102
Total CSV files for AfriVox: 91


In [39]:
def load_all_csv(lang_csv: List, drop_index: List = []) -> pd.DataFrame:
    df = pd.DataFrame()
    for csv_path in lang_csv:
        temp_df = pd.read_csv(csv_path)
        temp_df = temp_df.drop(drop_index)
        temp_df['model'] = map_model_to_csv(csv_path)
        df = pd.concat([df, temp_df])
    return df

def get_audio_files(df: pd.DataFrame, copy_dir: str = "error_samples/") -> None:
    audio_files = df['audio_path'].to_list()
    copy_dir = os.path.join(copy_dir, 'audio')
    os.makedirs(copy_dir, exist_ok=True)
    new_file_list = []
    for file in audio_files:
        try:
            shutil.copy2(file, copy_dir)
        except Exception as e:
            print(e)
            print(file)
        new_file_list.append(os.path.join(copy_dir, os.path.basename(file)))
    df['audio_path'] = new_file_list
    print(f"{len(audio_files)} copied to {copy_dir}")
    return df


In [10]:
def correct_common_voice(path):
    if 'common_voice_17' not in path:
        return path.replace("data/common_voice", "data/common_voice_17")
    else:
        return path



In [110]:
csv_file = 'results_2/open-source-full_test-gemini-2.0-flash-001-luganda-wer-0.8004-2181.csv'
df = pd.read_csv(csv_file)
all_wer = post_process_preds(df, non_english=True, remove_diacritics=True)
write_pred_inference_df(map_model_to_csv(csv_file), df, all_wer, language=map_language_to_csv(csv_file), split="full_test")
print(all_wer)

Cleanup WER: 79.81 %
TextNormalizer WER: 79.81 %
./results/open-source-full_test-gemini-2.0-flash-001-luganda-wer-0.7981-2181.csv
0.798058140870964


In [106]:
[x for x in os.listdir('results_2') if 'luganda' in x]

['open-source-full_test-gemini-2.0-flash-001-luganda-wer-0.8004-2181.csv',
 'open-source-full_test-gpt4o_preview-luganda-wer-1.3244-2181.csv',
 'open-source-full_test-facebook-mms-1b-all-luganda-wer-0.3052-2181.csv',
 'open-source-full_test-facebook-hf-seamless-m4t-large-luganda-wer-1.3662-2181.csv',
 'open-source-full_test-facebook-seamless-m4t-v2-large-luganda-wer-1.3522-2181.csv']

In [ ]:
results_v1 = os.listdir('results_2')
csv_files = [os.path.join("results_2", x) for x in results_v1]

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    all_wer = post_process_preds(df, non_english=True, remove_diacritics=True)
    write_pred_inference_df(map_model_to_csv(csv_file), df, all_wer, language=map_language_to_csv(csv_file), split="full_test")
    print(all_wer)

Cleanup WER: 246.68 %
TextNormalizer WER: 246.68 %
./results/open-source-full_test-gpt4o_preview-amharic-wer-2.4668-812.csv
2.4668447227850434
Cleanup WER: 72.11 %
TextNormalizer WER: 72.11 %
./results/open-source-full_test-gemini-2.0-flash-001-amharic-wer-0.7211-812.csv
0.7211272794253085
Cleanup WER: 266.66 %
TextNormalizer WER: 266.66 %
./results/open-source-full_test-whisper-large-v3-amharic-wer-2.6666-1280.csv
2.6665771523712336
Cleanup WER: 222.19 %
TextNormalizer WER: 222.19 %
./results/open-source-full_test-seamless-m4t-v2-large-shona-wer-2.2219-1553.csv
2.221926575815325
Cleanup WER: 107.84 %
TextNormalizer WER: 107.84 %
./results/open-source-full_test-gemini-2.0-flash-001-ga-wer-1.0784-75.csv
1.0784186649384315
Cleanup WER: 210.66 %
TextNormalizer WER: 210.66 %
./results/open-source-full_test-whisper-medium-yoruba-wer-2.1066-4716.csv
2.106644212223278
Cleanup WER: 85.91 %
TextNormalizer WER: 85.91 %
./results/open-source-full_test-hf-seamless-m4t-large-yoruba-wer-0.8591-4716.

./results/open-source-full_test-gpt4o_preview-xhosa-wer-1.3295-3335.csv
1.3295279428876938
Cleanup WER: 18.54 %
TextNormalizer WER: 18.54 %
./results/open-source-full_test-whisper-medium-french-wer-0.1854-1405.csv
0.1854189336235038
Cleanup WER: 12.86 %
TextNormalizer WER: 12.86 %
./results/open-source-full_test-whisper-large-v3-french-wer-0.1286-1405.csv
0.12858697341831182
Cleanup WER: 11.03 %
TextNormalizer WER: 11.03 %
./results/open-source-full_test-gemini-2.0-flash-001-french-wer-0.1103-1405.csv
0.11027514378983368
Cleanup WER: 46.23 %
TextNormalizer WER: 46.23 %
./results/open-source-full_test-whisper-large-v3-afrikaans-wer-0.4623-4341.csv
0.4623334397770488
Cleanup WER: 111.88 %
TextNormalizer WER: 111.88 %
./results/open-source-full_test-whisper-large-v3-shona-wer-1.1188-1553.csv
1.1187659559167236
Cleanup WER: 450.94 %
TextNormalizer WER: 450.94 %
./results/open-source-full_test-gpt4o_preview-ga-wer-4.5094-75.csv
4.509397278029812
Cleanup WER: 26.45 %
TextNormalizer WER: 26.4

In [21]:
results = os.listdir('results')
lang_csv = [os.path.join(result_dir, x) for x in results if "ga" in x and 'lu' not in x]
lang_csv

['results/open-source-full_test-gpt4o_preview-ga-wer-4.5094-75.csv',
 'results/open-source-full_test-gemini-2.0-flash-001-ga-wer-1.0784-75.csv']

In [23]:
df

,speaker_id,reference,audio_path,duration,source,language,gender,hypothesis,pred_clean,ref_clean,wer,hypothesis_clean,reference_clean,model
0,22,Nyɛɛ ekome,/data/fisd/ga/GaFm22-NBHk5JwO-Tmp079-ntRDLe.wav,2.880000,Ashesi-Org_Financial-Inclusion-Speech-Dataset,ga,female,که‌ایه کومه.,که‌ایه کومه,nyɛɛ ekome,1.000000,که‌ایه کومه,nyɛɛ ekome,gpt4o_preview
1,21,Adafitwaa kɛ shwɛmɔ,/data/fisd/ga/GaFm21-DAlkpH0k-Tmp077-JAWck2.wav,9.597812,Ashesi-Org_Financial-Inclusion-Speech-Dataset,ga,female,ادا فی چا کرشمہ۔,ادا فی چا کرشمہ,adafitwaa kɛ shwɛmɔ,1.333333,ادا فی چا کرشمہ,adafitwaa kɛ shwɛmɔ,gpt4o_preview
2,61,kɛfee lɛ tswaa mi,/data/fisd/ga/GaFm61-EAp3kOX6-Tmp102-hQvDjb.wav,7.837813,Ashesi-Org_Financial-Inclusion-Speech-Dataset,ga,female,Minä en oo.,mina en oo,kɛfee lɛ tswaa mi,1.000000,mina en oo,kɛfee lɛ tswaa mi,gpt4o_preview
3,20,Mɛni ofeɔ ŋmɛnɛ,/data/fisd/ga/GaMa20-BAt9gHqR-Tmp103-saWzSq.wav,7.353500,Ashesi-Org_Financial-Inclusion-Speech-Dataset,ga,male,Mini Ufuo Mene.,mini ufuo mene,mɛni ofeɔ ŋmɛnɛ,1.000000,mini ufuo mene,mɛni ofeɔ ŋmɛnɛ,gpt4o_preview
4,20,Ofainɛ noko ekoŋ yɛ ni otawɔ ni ofee aloo,/data/fisd/ga/GaMa20-JAxNmZd8-Tmp038-3r9eIe.wav,7.433500,Ashesi-Org_Financial-Inclusion-Speech-Dataset,ga,male,"O fainè, kòíkò n yè nínú ọ̀rọ̀ tó wá ní òfin àlò.",o faine koiko n ye ninu oro to wa ni ofin alo,ofainɛ noko ekoŋ yɛ ni otawɔ ni ofee aloo,1.222222,o faine koiko n ye ninu oro to wa ni ofin alo,ofainɛ noko ekoŋ yɛ ni otawɔ ni ofee aloo,gpt4o_preview
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,919fac7a-a4f2-48a2-b679-61e62d056f51,Mihu minyɛɛɛ maye nii gbɛkɛ ejaakɛ mimusu mli ...,/data/intron_mt/ga/35fd2bdc14ba88999b955c40fb2...,10.198000,intron-MT,ga,Female,"mihumi nyɛ mayɛning bɛkɛ, ɛgyaakɛ mi mmusumbri...",mihumi nyɛ mayɛning bɛkɛ ɛgyaakɛ mi mmusumbrii...,mihu minyɛɛɛ maye nii gbɛkɛ ejaakɛ mimusu mli ...,0.882353,mihumi nyɛ mayɛning bɛkɛ ɛgyaakɛ mi mmusumbrii...,mihu minyɛɛɛ maye nii gbɛkɛ ejaakɛ mimusu mli ...,gemini-2.0-flash-001
71,919fac7a-a4f2-48a2-b679-61e62d056f51,Fɛ̃i hu miyeɔ lɛ shi ebɛ atridii. Nɔni jiɔɔ kp...,/data/intron_mt/ga/69a7d3c4f10aab89de1c556ae3a...,13.893000,intron-MT,ga,Female,"fa yi hu mi yoo le, sin e be a tri-di yi, ni w...",fa yi hu mi yoo le sin e be a tri di yi ni won...,fɛi hu miyeɔ lɛ shi ebɛ atridii nɔni jiɔɔ kpo ...,1.291667,fa yi hu mi yoo le sin e be a tri di yi ni won...,fɛi hu miyeɔ lɛ shi ebɛ atridii nɔni jiɔɔ kpo ...,gemini-2.0-flash-001
72,919fac7a-a4f2-48a2-b679-61e62d056f51,"Gbi nyɔŋma kɛ enumɔ sɛɛ mli lɛ,gbee lɛ gbo. Es...",/data/intron_mt/ga/e4764a52ebe0523f67ee78c6b33...,9.119000,intron-MT,ga,Female,"diinyun makhɛ ɛnnumɔ sɛm ninnle bɛlɛgbo, ɛsaan...",diinyun makhɛ ɛnnumɔ sɛm ninnle bɛlɛgbo ɛsaani...,gbi nyɔŋma kɛ enumɔ sɛɛ mli lɛ gbee lɛ gbo esa...,1.000000,diinyun makhɛ ɛnnumɔ sɛm ninnle bɛlɛgbo ɛsaani...,gbi nyɔŋma kɛ enumɔ sɛɛ mli lɛ gbee lɛ gbo esa...,gemini-2.0-flash-001
73,919fac7a-a4f2-48a2-b679-61e62d056f51,Mi datrɛfonyo bi mi akɛ miyɛ atridii yɛ beefio...,/data/intron_mt/ga/eb7cd16a9b917af8790745d18c7...,14.523000,intron-MT,ga,Female,Me daa twerefoɔ no bi mmiɛnsa akɛseɛ mmienu a ...,me daa twerefoɔ no bi mmiɛnsa akɛseɛ mmienu a ...,mi datrɛfonyo bi mi akɛ miyɛ atridii yɛ beefio...,1.200000,me daa twerefoɔ no bi mmiɛnsa akɛseɛ mmienu a ...,mi datrɛfonyo bi mi akɛ miyɛ atridii yɛ beefio...,gemini-2.0-flash-001


In [41]:
results = os.listdir('results')
for language in tqdm(language_list, total=len(language_list), colour='green'):
    if language.lower() == 'ga':
        lang_csv = [os.path.join(result_dir, x) for x in results if language.lower() in x and 'lu' not in x]
    else:
        lang_csv = [os.path.join(result_dir, x) for x in results if language.lower() in x]
    if language.lower() == 'yoruba':
        drop_index_list = [514, 2354, 2555, 2800, 3255, 4125, 2427, 3253, 1263, 2373, 3603, 2769, 3592, 3590, 3588]
    else:
        drop_index_list = []
    try:
        df = load_all_csv(lang_csv, drop_index=drop_index_list)
        df['audio_path'] = df['audio_path'].apply(lambda x: x.replace("/home/busayo/busayo/mls_benchmark/", "/"))
        df['audio_path'] = df['audio_path'].apply(correct_common_voice)
    except Exception as e:
        print(e)
        print(language)
        print(df.columns)
        break

    errors = sample_weighted_by_wer(df, n_samples=50)

    copy_dir = f"error_samples/{language}/"
    os.makedirs(copy_dir, exist_ok=True)
    df = get_audio_files(errors, copy_dir)

    df.sort_values(by='wer', ascending=False).to_csv(f"{copy_dir}{language}_errors.csv", index=False)
    print(f"Audio and csv created for {language}")




  5%|▌         | 1/20 [00:00<00:09,  1.96it/s]

50 copied to error_samples/yoruba/audio
Audio and csv created for yoruba


 10%|█         | 2/20 [00:00<00:07,  2.49it/s]

50 copied to error_samples/hausa/audio
Audio and csv created for hausa


 25%|██▌       | 5/20 [00:01<00:02,  5.66it/s]

...

50 copied to error_samples/swahili/audio
Audio and csv created for swahili
50 copied to error_samples/xhosa/audio
Audio and csv created for xhosa


 90%|█████████ | 18/20 [00:03<00:00,  6.52it/s]

50 copied to error_samples/zulu/audio
Audio and csv created for zulu
50 copied to error_samples/sesotho/audio
Audio and csv created for sesotho
50 copied to error_samples/tswana/audio
Audio and csv created for tswana


100%|██████████| 20/20 [00:04<00:00,  4.87it/s]

50 copied to error_samples/french/audio
Audio and csv created for french
50 copied to error_samples/kinyarwanda/audio
Audio and csv created for kinyarwanda


In [69]:
lang_csv = [os.path.join(result_dir, x) for x in results if 'yoruba'.lower() in x ]

df = load_all_csv(lang_csv)

errors = sample_weighted_by_wer(df, n_samples=300)

In [70]:
errors.sort_values(by='wer', ascending=False)['model'].value_counts()

gemini-2.0-flash-001     50
whisper-medium           50
seamless-m4t-v2-large    50
mms-1b-all               50
whisper-large-v3         50
gpt4o_preview            50
Name: model, dtype: int64

In [71]:
errors.audio_path.nunique()

287

In [72]:
errors.sort_values(by='wer', ascending=False)['source'].value_counts()

naijavoices     126
intron-MT        90
common_voice     48
fleurs           36
Name: source, dtype: int64

In [73]:
errors['wer'].min()

0.2142857142857142

In [ ]:
language_list

In [34]:
index = 0
df = pd.read_csv(f"/home/busayo/busayo/asr_benchmarking/error_samples/{language_list[index]}/{language_list[index]}_errors.csv")
df

,speaker_id,speaker_id.1,audio_path,reference,hypothesis,pred_clean,ref_clean,wer,source,model
0,NaN,eb17abb948d9d4e5a45de18b277b543139e5e3628fb0b4...,error_samples/yoruba/audio/common_voice_yo_365...,Iṣẹ́ ni àwọn eléré ẹ̀fẹ̀ máa ń ṣe.,Iṣẹ mi àwọn ilé iṣẹ́ mi àwọn ilé iṣẹ́ mi àw...,ise mi awon ile ise mi awon ile ise mi awon il...,ise ni awon elere efe maa n se,372.125000,common_voice,gemini-2.0-flash-001
1,52442.0,NaN,error_samples/yoruba/audio/20240224093304-130-...,Má ṣe fún àwọn ọmọdé ní kooro moringa fún jíjẹ.,ma je fun o mo n ko mo mo mo mo mo mo mo mo m...,ma je fun o mo n ko mo mo mo mo mo mo mo mo m...,ma se fun awon omode ni kooro moringa fun jije,25.400000,naijavoices,seamless-m4t-v2-large
2,2670.0,NaN,error_samples/yoruba/audio/20240128091001-138-...,Àwọn obìnrin náà ń jẹ iṣu.,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,awon obinrin naa n je isu,24.666667,naijavoices,whisper-medium
3,52000.0,NaN,error_samples/yoruba/audio/20240128091001-138-...,Àwọn obìnrin náà ń jẹ iṣu.,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,awon obinrin naa n je isu,24.666667,naijavoices,whisper-medium
4,52453.0,NaN,error_samples/yoruba/audio/20240311010856-76-5...,Ó rà ẹ̀gbà ọrùn fún mi gẹ́gẹ́ bí ẹ̀bùn ọjọ́ ìbí,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,o ra egba orun fun mi gege bi ebun ojo ibi,13.454545,naijavoices,whisper-medium
5,47569.0,NaN,error_samples/yoruba/audio/eb82f6e2f61b4f49ec7...,"Kò tí ì dára si, ṣùgbọ́n.... kò sì tí ì burú s...",ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,ko ti i dara si sugbon ko si ti i buru si i,11.384615,intron-MT,whisper-medium
6,5651.0,NaN,error_samples/yoruba/audio/1763226308360520016...,ọ̀rọ̀ tí à ń pè ní ọ̀làjú wá látí inú èdè ìlú ...,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប ប...,oro ti a n pe ni olaju wa lati inu ede ilu lat...,4.384615,fleurs,whisper-medium
7,31378.0,NaN,error_samples/yoruba/audio/common_voice_yo_365...,Ọmọlúwàbí ṣọ̀wọ́n láàárín àwọn òlóṣèlú Nàìj...,ò m ù g ọ ab ís ò gun lá àr ín àwọn ará ìlú n...,o m u g o ab is o gun la ar in awon ara ilu n...,omoluwabi sowon laaarin awon oloselu naijiria,3.333333,common_voice,seamless-m4t-v2-large
8,NaN,EYW1T,error_samples/yoruba/audio/20240128091001-138-...,Àwọn obìnrin náà ń jẹ iṣu.,"Malheureusement, je ne peux pas répondre à des...",malheureusement je ne peux pas repondre a des ...,awon obinrin naa n je isu,3.166667,naijavoices,gpt4o_preview
9,NaN,PBLQ4,error_samples/yoruba/audio/20240308022037-135-...,Ṣe o ri gbẹnagbẹna loni?,"Ok, provide the audio file, and I will transcr...",ok provide the audio file and i will transcrib...,se o ri gbenagbena loni,2.600000,naijavoices,gpt4o_preview


## Language Identification

In [3]:
df = pd.read_csv(f"data/mls_data_index.csv")    

In [10]:
from transformers import Wav2Vec2ForSequenceClassification, AutoFeatureExtractor
import torch
import librosa 

model_id = "facebook/mms-lid-256"

processor = AutoFeatureExtractor.from_pretrained(model_id)
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_id).cuda()


In [15]:
def identify_language(df):

    for idx, row in df.iterrows():
        audio_path = row['audio_path']
        audio, sr = librosa.load(audio_path, sr=16000)
        inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True).to('cuda')
        
        with torch.no_grad():
            logits = model(**inputs).logits
            predicted_ids = torch.argmax(logits, dim=-1)
        
        df.loc[idx, 'audio_language'] = model.config.id2label[predicted_ids.item()]
    return df

In [16]:
identify_language(df.head(2))

/var/tmp/ipykernel_215452/1036966514.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[idx, 'audio_language'] = model.config.id2label[predicted_ids.item()]


,speaker_id,text,audio_path,duration,source,language,gender,audio_language
0,Q7FW1,Ọmọ ọ̀jọ̀gbọ́n fẹ̀yìntì ni mò ń fẹ́,/data/naijavoices/yoruba/20240326111846-233-43...,2.28,naijavoices,yoruba,female,yor
1,VJA5Q,Mamman Shata ɗan jihar Gombe ne,/data/naijavoices/hausa/20240216132916-162-130...,2.04,naijavoices,hausa,female,hau


## Interspeech

**Selected Languages**

> **Baseline**: english, french  
> **African Languages**: akan, arabic, french, hausa, kinyarwanda, shona, swahili, xhosa, yoruba, zulu.

In [6]:
%cd ../

/home/busayo/busayo/asr_benchmarking


In [1]:
languages = ['akan', 'arabic', 'french', 'hausa', 'kinyarwanda', 'shona', 'swahili', 'xhosa', 'yoruba', 'zulu']

In [34]:
models = ['whisper-large-v3', 'whisper-medium', 'gemini-2.0-flash-001', 'gpt4o_preview', 'seamless-m4t-v2-large', 'canary-1b', 'mms-1b-all', 'qwen2-audio-7b-instruct']

In [37]:
result_dir = "results/"

# For example, let's assume result_list contains all files in the results directory:
result_list = [os.path.join(result_dir, x) for x in os.listdir(result_dir)]
print(f"Before filtering: {len(result_list)}")


Before filtering: 102


In [35]:
# Check each file path if it contains any of the strings in models or languages:
result_list = [x for x in result_list if any(model in x for model in models) and any(language in x for language in languages)]

print(f"After filtering: {len(result_list)}")

selected = {map_language_to_csv(x) for x in result_list}
len(selected), selected

After filtering: 53


(10,
 {'akan',
  'arabic',
  'french',
  'hausa',
  'kinyarwanda',
  'shona',
  'swahili',
  'xhosa',
  'yoruba',
  'zulu'})

In [36]:
for model in models:
    print(f"Model: {model}")
    
    df = pd.DataFrame()
    model_results = [x for x in result_list if model in x]
    model_lang = []
    for result in model_results:
        temp_df = pd.read_csv(result)
        model_lang.append(map_language_to_csv(result))
        df = pd.concat([df, temp_df])

    all_wer = post_process_preds(df, non_english=True, remove_diacritics=True)
    print(f"Languages: {model_lang}")
    print(f"Absolute WER: {all_wer}")
    print()
    print("---"*20)
    print()

Model: whisper-large-v3


Cleanup WER: 74.04 %
TextNormalizer WER: 74.04 %
Languages: ['arabic', 'shona', 'shona', 'hausa', 'french', 'arabic', 'swahili', 'yoruba']
Absolute WER: 0.7403656283839735

------------------------------------------------------------

Model: whisper-medium
Cleanup WER: 138.62 %
TextNormalizer WER: 138.62 %
Languages: ['shona', 'yoruba', 'arabic', 'french', 'hausa', 'swahili']
Absolute WER: 1.3862138058382116

------------------------------------------------------------

Model: gemini-2.0-flash-001
Cleanup WER: 49.91 %
TextNormalizer WER: 49.91 %
Languages: ['zulu', 'arabic', 'yoruba', 'swahili', 'kinyarwanda', 'french', 'hausa', 'shona', 'xhosa', 'akan']
Absolute WER: 0.4991427156890846

------------------------------------------------------------

Model: gpt4o_preview
Cleanup WER: 97.23 %
TextNormalizer WER: 97.23 %
Languages: ['akan', 'yoruba', 'swahili', 'shona', 'french', 'arabic', 'zulu', 'hausa', 'kinyarwanda', 'xhosa']
Absolute WER: 0.9722672434009954

--------------------------

**All languages**

In [38]:
for model in models:
    print(f"Model: {model}")
    
    df = pd.DataFrame()
    model_results = [x for x in result_list if model in x]
    model_lang = []
    for result in model_results:
        temp_df = pd.read_csv(result)
        model_lang.append(map_language_to_csv(result))
        df = pd.concat([df, temp_df])

    all_wer = post_process_preds(df, non_english=True, remove_diacritics=True)
    print(f"Languages: {model_lang}")
    print(f"Absolute WER: {all_wer}")
    print()
    print("---"*20)
    print()

Model: whisper-large-v3


Cleanup WER: 84.01 %
TextNormalizer WER: 84.01 %
Languages: ['arabic', 'shona', 'amharic', 'amharic', 'shona', 'afrikaans', 'hausa', 'french', 'arabic', 'swahili', 'yoruba']
Absolute WER: 0.8401181252381558

------------------------------------------------------------

Model: whisper-medium
Cleanup WER: 151.20 %
TextNormalizer WER: 151.20 %
Languages: ['amharic', 'shona', 'afrikaans', 'yoruba', 'arabic', 'french', 'hausa', 'swahili']
Absolute WER: 1.5120450200912885

------------------------------------------------------------

Model: gemini-2.0-flash-001
Cleanup WER: 59.75 %
TextNormalizer WER: 59.75 %
Languages: ['zulu', 'afrikaans', 'arabic', 'yoruba', 'luganda', 'swahili', 'sesotho', 'kinyarwanda', 'igbo', 'pedi', 'french', 'amharic', 'hausa', 'shona', 'xhosa', 'twi', 'tswana', 'ga', 'akan', 'fulani']
Absolute WER: 0.5975444094483214

------------------------------------------------------------

Model: gpt4o_preview
Cleanup WER: 109.61 %
TextNormalizer WER: 109.61 %
Languages: ['ak

In [6]:
language = 'hausa'
csv_file = f"/home/busayo/busayo/asr_benchmarking/error_samples/{language}/{language}_errors.csv"
df = pd.read_csv(csv_file)

In [7]:
def get_text(row):
    print(f"Ref: {row['reference'].iloc[0]}")
    print(f"Pred: {row['hypothesis'].iloc[0]}")
    print(f"Model: {row['model'].iloc[0]}")
    print(f"WER: {row['wer']}")

In [8]:
file_ = 'error_samples/hausa/audio/20240112152329-157-1280-272226-a-al-ada-hausawa-ana-yiwa-yara.wav'
get_text(df[df['audio_path'] == file_])

Ref: A al'ada hausawa ana yiwa yara maza kaciya in su ka kai shekara bakwai
Pred: A daddare Hausawa ana yiwa yara maza kaciya in sun kai shekara bakwai.
Model: gemini-2.0-flash-001
WER: 47    0.266667
Name: wer, dtype: float64


In [24]:
Audio(file_)